In [ ]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack 2.0 - Introduction

This tutorial gives an overview of how to use DeepTrack 2.0.

### What is DeepTrack 2.0?

DeepTrack 2.0 is a software that provides a comprehensive framework for digital microscopy enhanced by deep learning. Possible applications include particle identification, particle tracking, image segmentation, and cell counting. This tutorial focuses on the simple task to track a particle.

## 1. Generate images

In order to train and validate deep-learning models to be used in digital microcopy, we need to generate synthetic images. For example, these images may be like the output of a digital video micoscopy experiment, where we want to track some particles.

In DeepTrack 2.0, any image can be viewed as a series of **features** applied in a sequence. Each feature takes an input image and updates it according to an **update rule**. For example, a feature can add a particle, introduce some noise, or image something through an optical device.

### Features and properties

In DeepTrack 2.0, these features are classes implementing the class `Feature` (see also [features_example](../examples/features_example.ipynb)). The way a feature updates an image is determined by the values passed to the class constructor. These inputs are converted to **properties** (see also [properties_example](../examples/properties_example.ipynb)). For example, a property could be the position of a particle. 

For example, the code below creates a feature: a point particle implemented by the class `PointParticle` with properties `position=(0, 0)`, `position_unit="pixel"`, and `intensity=1`.

In [ ]:
from deeptrack.scatterers import PointParticle

particle = PointParticle(
    position=(0, 0),
    position_unit="pixel", # the default is meter
    intensity=1
)

The above feature is completely deterministic so that the particle will always be at position (0, 0). For machine learning, it may be more useful to add a particle at a random position. This can be done by passing a lambda function that returns a pair of random numbers to the property `position`.

In [ ]:
import numpy as np

IMAGE_SIZE = 64

particle = PointParticle(
    position=lambda: np.random.rand(2) * IMAGE_SIZE,
    position_unit="pixel",
    intensity=1
)

A point particle on its own does not make much sense when thinking of actual experiments. In order to get a more realistic image, we need to define also the optical device through which it is viewed. In DeepTrack 2.0, optical devices are features that convolve the input image with a pupil function. Here, we will use a fluorescence microscope `fluorescence_microscope`, which is implemented by the feature `Fluorescence` (see also [optics_example](../examples/optics_example.ipynb)). 

By calling the fluorescence microscope `fluorescence_microscope` with the point particle `particle`, we create a new feature which resolves the image of the particle as seen through the fluorescence microscope.

In [ ]:
from deeptrack.optics import Fluorescence

fluorescence_microscope = Fluorescence(
    NA=0.8,
    wavelength=680e-9,
    magnification=10,
    resolution=1e-6,
    output_region=(0, 0, IMAGE_SIZE, IMAGE_SIZE)
)

imaged_particle = fluorescence_microscope(particle)

We can finally create the image of the particle by calling the method `imaged_particle.resolve()` and plot it.

In [ ]:
output_image = imaged_particle.resolve()

import matplotlib.pyplot as plt
plt.imshow(np.squeeze(output_image), cmap='gray')
plt.show()

You might have noticed that, even though the position of the particle is random, it doesn't change if you refresh the cell. This is because the value of the particle position is stored as a property of the particle, useful for example to know the ground truth value of the particle position to train a deep-learning model.

If you want the particle position to change each time you refresh the cell, you need to call the method `imaged_particle.update()` before resolving the image.

In [ ]:
imaged_particle.update()
output_image = imaged_particle.resolve()

plt.imshow(np.squeeze(output_image), cmap='gray')
plt.show()

### Adding more features

Features can be combined (see also [features_example](../examples/features_example.ipynb)) using overloaded operators (+, \*, \*\* and ()). Here exemplify the add operator (+) and the power operator (\*\*).

In [ ]:
# The + operator

particle_1 = PointParticle(
    position=lambda: np.random.rand(2) * IMAGE_SIZE,
    position_unit="pixel",
    intensity=1
)

particle_2 = PointParticle(
    position=lambda: np.random.rand(2) * IMAGE_SIZE,
    position_unit="pixel",
    intensity=1
)

two_particles = particle_1 + particle_2

# imaged_two_particles is a new feature that first resolves particle_1 and then particle_2, then images it
imaged_two_particles = fluorescence_microscope(two_particles)

output_image = imaged_two_particles.resolve()

plt.imshow(np.squeeze(output_image), cmap='gray')
plt.show()

In [ ]:
# The ** operator

particle = PointParticle(
    position=lambda: np.random.rand(2) * IMAGE_SIZE,
    position_unit="pixel",
    intensity=1
)

five_particles = particle**5

# five_particles is a feature that resolves five deep copies of particle, then images it
imaged_five_particles = fluorescence_microscope(five_particles)

output_image = imaged_five_particles.resolve()

plt.imshow(np.squeeze(output_image), cmap='gray')
plt.show()

### Adding noise

To make the image more realistic, we can now add some noise (see also [noises_example](../examples/noises_example.ipynb)).

In [ ]:
from deeptrack.noises import Offset, Poisson

# Adds a constant value to the background
offset = Offset(offset=0.01)

# Introduce Poisson noise to the image
poisson_noise = Poisson(snr=100)

# noisy_particles resolves five particles, then adds a offset, images it, then introduces poisson noise
imaged_noisy_particles = fluorescence_microscope(particle**5) + offset + poisson_noise

output_image = imaged_noisy_particles.resolve()

plt.imshow(np.squeeze(output_image), cmap='gray')
plt.show()

## 2. Retrieve information about the image

To train a supervised machine-learning model, labelled images are needed. When a feature is resolved, it automatically stores the properties of all features used to create the image. This allows us to extract information about the image, for example, in order to use them to train machine-learning models.

Here, we extract the position of all the particles and plot them as red crosses on the generated image.

In [ ]:
def get_positions(image):
    # All properties are stored in the `properties` field of the output.
    positions = [property_dict["position"] for property_dict in image.properties if "position" in property_dict]
    return np.array(positions)


imaged_noisy_particles.update()
output_image = imaged_noisy_particles.resolve()

positions = get_positions(output_image)

plt.imshow(np.squeeze(output_image), cmap='gray')
plt.scatter(positions[:, 1], positions[:, 0], c="r", marker="x")
plt.show()

## 3. Wrap features in generators

Generators are ways to continuously resolve new images, and are the prefered interface to machine learning models. The default generator is defined in the module `generators` (see also [generators_example](../examples/generators_example.ipynb)). We can also optionally pass a label function that will be called on every image, in this case we will use the function `get_positions` that we have implemented above.

In [ ]:
from deeptrack.generators import Generator

generator = Generator().generate(imaged_noisy_particles, label_function=get_positions)

for _ in range(4):
    # Outputs shape (1, height, width, 1)
    next_image, positions = next(generator)
    plt.imshow(np.squeeze(next_image), cmap='gray')
    plt.scatter(positions[0, :, 1], positions[0, :, 0], c='r', marker='x')
    plt.show()

## 4. Training the model

We are finally ready to create a deep-learning model to track the particles.
We will use a convolutional neural network implemented by the function `convolutional` (see also [models_example](../examples/models_example.ipynb)) to track point particles (`particle`) imaged through a fluorescence microscope (`fluorescence_microscope`) with some noise (`offset` and `poisson_noise`). We will furthermore use the feature `NormalizeMinMax` to normalize the dynamic range of the images btween 0 and 1.

This model is designed to track a single particle with a high accuracy. It is not well suited for the task of multi-particle tracking.

In [ ]:
from deeptrack.models import convolutional
from deeptrack.math import NormalizeMinMax

# DEFINE LABEL FUNCTION
def get_position(image):
    for propertydict in image.properties:
        if "position" in propertydict:
            return propertydict["position"] / IMAGE_SIZE
        
# DEFINE MODEL
model = convolutional(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1), number_of_outputs=2)



# DEFINE TRAINING SET
normalization = NormalizeMinMax(min=0, max=1)
training_set = fluorescence_microscope(particle) + offset + poisson_noise + normalization

# DEFINE GENERATOR
generator = Generator().generate(training_set, get_position, batch_size=32)

Now we train the model. Be patient, this might take some time (several minutes).

In [ ]:
# TRAIN TRACKER
model.fit(generator, epochs=100, steps_per_epoch=10)

Finally, we test the trained model on some newly generated images.

In [ ]:
images, real_positions = next(generator)

measured_positions = model.predict(images)

for i in range(images.shape[0]):
    
    image = np.squeeze(images[i])
    
    measured_position_x = measured_positions[i, 1] * IMAGE_SIZE
    measured_position_y = measured_positions[i, 0] * IMAGE_SIZE

    real_position_x = real_positions[i, 1] * IMAGE_SIZE
    real_position_y = real_positions[i, 0] * IMAGE_SIZE

    plt.imshow(image, cmap='gray')
    plt.scatter(real_position_x, real_position_y, s=70, c='r', marker='x')
    plt.scatter(measured_position_x, measured_position_y, s=100, marker='o', facecolor='none', edgecolors='b')
    plt.show()